In [119]:
#ライブラリのインポート
from bs4 import BeautifulSoup
import re #不要文字削除
import requests
from time import sleep
import time
from tqdm import tqdm   #for文の進捗確認
import pandas as pd

In [120]:
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&cb=0.0&ct=20.0&md=05&md=06&md=07&md=08&md=09&et=10&mb=0&mt=9999999&cn=15&tc=0400301&fw2="
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
maxpage = int(soup.select("ol.pagination-parts a")[-1].text)

In [121]:

# 正常にHTML情報が取得できれば以下のコードを実行
if res.status_code == 200:

    names = []
    addresses = []

    access1 = []
    access2 = []
    access3 = []

    access1_walk = []
    access2_walk = []
    access3_walk = []

    ages = []
    maxfloors = []

    floors = []
    rents = []
    layouts = []
    sizes = []
    image_urls = []

    for x in range(maxpage):
        res = requests.get(url+"&page="+str(x))
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")
        suumo_cassetteitem_from_html = soup.select('div.cassetteitem')
        time.sleep(1)

        for sc in suumo_cassetteitem_from_html:
            suumo_name = sc.select('div.cassetteitem_content-title')[0].text
            suumo_address = sc.select("li.cassetteitem_detail-col1")[0].text

            suumo_access = sc.select('li.cassetteitem_detail-col2')[0].text.replace('\n', '').strip(',')
            access_parts = re.findall(r'([^\d]+?/\S+?駅)\s+(\S+分)', suumo_access)
            access_parts += [('', '')] * (3 - len(access_parts))  # 不足分を空のタプルで埋める

            suumo_age, suumo_maxfloor = sc.select("li.cassetteitem_detail-col3")[0].text.split()
            if suumo_age == "新築":
                suumo_age = "0"
            else:
                suumo_age = suumo_age.replace("築", "").replace("年", "")

            max_floor_number = re.search(r'(\d+)階建', suumo_maxfloor)
            suumo_maxfloor = max_floor_number.group(1) if max_floor_number else "0"  # マッチしない場合は"0"

            tbody_items = sc.select('tbody')

            for ti in tbody_items:
                suumo_floor = ti.select('tr.js-cassette_link')[0].text.split()[0]
                suumo_rent = ti.select('span.cassetteitem_other-emphasis, span.ui-text--bold')[0].text
                suumo_layout = ti.select('span.cassetteitem_madori')[0].text
                suumo_size = ti.select('span.cassetteitem_menseki')[0].text

                suumo_floor = suumo_floor.replace('階', '')
                suumo_rent = suumo_rent.replace('万円', '')
                suumo_size = suumo_size.replace('m2', '')

                names.append(suumo_name)
                addresses.append(suumo_address)
                ages.append(suumo_age)
                maxfloors.append(suumo_maxfloor)
                floors.append(suumo_floor)
                rents.append(suumo_rent)
                layouts.append(suumo_layout)
                sizes.append(suumo_size)

                # 各物件ごとにアクセス情報と徒歩情報を追加
                access1_station, access1_walk_time = access_parts[0]
                access2_station, access2_walk_time = access_parts[1]
                access3_station, access3_walk_time = access_parts[2]

                access1.append(access1_station)
                access2.append(access2_station)
                access3.append(access3_station)

                access1_walk.append(access1_walk_time.replace('歩', '').replace('分', ''))
                access2_walk.append(access2_walk_time.replace('歩', '').replace('分', ''))
                access3_walk.append(access3_walk_time.replace('歩', '').replace('分', ''))
                
                #js-bukkenList > ul:nth-child(2) > li:nth-child(2) > div > div.cassetteitem-detail
                img_tag = sc.select_one('img.js-noContextMenu.js-linkImage.js-adjustImg')
                image_url = img_tag['rel'] if img_tag and 'rel' in img_tag.attrs else ''
                image_urls.append(image_url)
                #img_tag = sc.select('js-noContextMenu js-linkImage js-scrollLazy js-adjustImg')
                #image_url = img_tag['rel'] if img_tag and img_tag.has_attr('rel') else '画像のURLが見つかりません'
                #image_urls.append(image_url)

    data = {
        'Name': names,
        'Address': addresses,
        'Access1': access1,
        'Access2': access2,
        'Access3': access3,
        'Access1_Walk': access1_walk,
        'Access2_Walk': access2_walk,
        'Access3_Walk': access3_walk,
        'Age': ages,
        'Max_Floor': maxfloors,
        'Floor': floors,
        'Rent': rents,
        'Layout': layouts,
        'Size': sizes,
        'Image_URL':image_urls,
    }


In [122]:
for key in data.keys():
    print(f"Length of {key}: {len(data[key])}")


Length of Name: 4735
Length of Address: 4735
Length of Access1: 4735
Length of Access2: 4735
Length of Access3: 4735
Length of Access1_Walk: 4735
Length of Access2_Walk: 4735
Length of Access3_Walk: 4735
Length of Age: 4735
Length of Max_Floor: 4735
Length of Floor: 4735
Length of Rent: 4735
Length of Layout: 4735
Length of Size: 4735
Length of Image_URL: 4735


In [123]:
for key in data.keys():
    print(f"Length of {key}: {len(data[key])}")

In [124]:
df = pd.DataFrame(data)

Name            object
Address         object
Access1         object
Access2         object
Access3         object
Access1_Walk    object
Access2_Walk    object
Access3_Walk    object
Age             object
Max_Floor       object
Floor           object
Rent            object
Layout          object
Size            object
Image_URL       object
dtype: object

In [125]:
#データ型の確認
df.dtypes

,Name,Address,Access1,Access2,Access3,Access1_Walk,Access2_Walk,Access3_Walk,Age,Max_Floor,Floor,Rent,Layout,Size,Image_URL
0,sunrise 浅草田原町,東京都台東区西浅草１,東京メトロ銀座線/田原町駅,つくばエクスプレス/浅草駅,都営浅草線/浅草駅,1,3,8,2,10,7,17.1,2K,41.68,https://img01.suumo.com/front/gazo/fr/bukken/8...
1,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,4,14.4,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...
2,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,6,14.5,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...
3,パークアクシス押上レジデンス,東京都墨田区向島３,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,5,16.7,2DK,42.21,https://img01.suumo.com/front/gazo/fr/bukken/7...
4,パークアクシス押上レジデンス,東京都墨田区向島３,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,4,19.5,2LDK,50.34,https://img01.suumo.com/front/gazo/fr/bukken/7...


In [126]:
df

In [135]:
df.drop_duplicates(subset=['Name','Address', 'Floor', 'Layout', 'Size'], inplace=True)
df.head(
)

In [136]:
df.duplicated()

In [137]:
#google spread sheets 出力
#ライブラリのインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#環境変数関連
from dotenv import load_dotenv
load_dotenv()
import os

/var/folders/kg/8x8shztx3cxdm81q2gqnhjf40000gn/T/ipykernel_88869/3352757521.py:6: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("A1", values)


{'spreadsheetId': '1PfzOsRARhloJ3ebywaTIW4yj_17b7koLv2mv7okRlrE',
 'updatedRange': 'db!A1:O2315',
 'updatedRows': 2315,
 'updatedColumns': 15,
 'updatedCells': 34725}

In [138]:
#スコープとjsonファイルを使って認証情報を取得
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
#SERVICE_ACCOUNT_FILE = '/Users/maedaryo/step3/tech0-step3-407115-94fd3b930a55.json'
SERVICE_ACCOUNT_FILE = os.getenv("SERVICE_ACCOUNT_FILE")
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

In [139]:
#認証情報をauthorize関数に渡してスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

#シート情報を取得して変数に代入
SPREADSHEET_KEY = os.getenv("SPREADSHEET_KEY")
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("db")

('sunrise 浅草田原町', '東京都台東区西浅草１', '東京メトロ銀座線/田原町駅', 'つくばエクスプレス/浅草駅', '都営浅草線/浅草駅', '1', '3', '8', '2', '10', '7', '17.1', '2K', '41.68', 'https://img01.suumo.com/front/gazo/fr/bukken/897/100319979897/100319979897_gw.jpg')
('東京メトロ南北線 四ツ谷駅 11階建 築2年', '東京都新宿区左門町', '東京メトロ南北線/四ツ谷駅', '東京メトロ丸ノ内線/四谷三丁目駅', 'ＪＲ中央線/信濃町駅', '14', '3', '7', '2', '11', '4', '14.4', '2K', '26.04', 'https://img01.suumo.com/front/gazo/fr/bukken/027/100361959027/100361959027_gw.jpg')
('東京メトロ南北線 四ツ谷駅 11階建 築2年', '東京都新宿区左門町', '東京メトロ南北線/四ツ谷駅', '東京メトロ丸ノ内線/四谷三丁目駅', 'ＪＲ中央線/信濃町駅', '14', '3', '7', '2', '11', '6', '14.5', '2K', '26.04', 'https://img01.suumo.com/front/gazo/fr/bukken/027/100361959027/100361959027_gw.jpg')
('パークアクシス押上レジデンス', '東京都墨田区向島３', '東武伊勢崎線/とうきょうスカイツリー駅', '都営浅草線/本所吾妻橋駅', '東京メトロ半蔵門線/押上駅', '6', '10', '10', '6', '7', '5', '16.7', '2DK', '42.21', 'https://img01.suumo.com/front/gazo/fr/bukken/703/100362266703/100362266703_gw.jpg')
('パークアクシス押上レジデンス', '東京都墨田区向島３', '東武伊勢崎線/とうきょうスカイツリー駅', '都営浅草線/本所吾妻橋駅', '東京メトロ半蔵門線/押上駅', '6'

In [140]:
#特定の列（'Address', 'Floor', 'Rent', 'Layout', 'Size'）に基づいて重複する行を削除
df.drop_duplicates(subset=['Address', 'Floor', 'Rent', 'Layout', 'Size'], inplace=True)

values = [df.columns.values.tolist()] + df.values.tolist()

worksheet.update("A1", values)

/var/folders/kg/8x8shztx3cxdm81q2gqnhjf40000gn/T/ipykernel_88869/3352757521.py:6: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("A1", values)


{'spreadsheetId': '1PfzOsRARhloJ3ebywaTIW4yj_17b7koLv2mv7okRlrE',
 'updatedRange': 'db!A1:O2315',
 'updatedRows': 2315,
 'updatedColumns': 15,
 'updatedCells': 34725}

In [141]:
#SQLite
import sqlite3
conn = sqlite3.connect('fudosan.db')
df.to_sql('suumo', conn, if_exists='replace', index=False)
conn.close()


In [142]:
import sqlite3
conn = sqlite3.connect('fudosan.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM suumo")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('sunrise 浅草田原町', '東京都台東区西浅草１', '東京メトロ銀座線/田原町駅', 'つくばエクスプレス/浅草駅', '都営浅草線/浅草駅', '1', '3', '8', '2', '10', '7', '17.1', '2K', '41.68', 'https://img01.suumo.com/front/gazo/fr/bukken/897/100319979897/100319979897_gw.jpg')
('東京メトロ南北線 四ツ谷駅 11階建 築2年', '東京都新宿区左門町', '東京メトロ南北線/四ツ谷駅', '東京メトロ丸ノ内線/四谷三丁目駅', 'ＪＲ中央線/信濃町駅', '14', '3', '7', '2', '11', '4', '14.4', '2K', '26.04', 'https://img01.suumo.com/front/gazo/fr/bukken/027/100361959027/100361959027_gw.jpg')
('東京メトロ南北線 四ツ谷駅 11階建 築2年', '東京都新宿区左門町', '東京メトロ南北線/四ツ谷駅', '東京メトロ丸ノ内線/四谷三丁目駅', 'ＪＲ中央線/信濃町駅', '14', '3', '7', '2', '11', '6', '14.5', '2K', '26.04', 'https://img01.suumo.com/front/gazo/fr/bukken/027/100361959027/100361959027_gw.jpg')
('パークアクシス押上レジデンス', '東京都墨田区向島３', '東武伊勢崎線/とうきょうスカイツリー駅', '都営浅草線/本所吾妻橋駅', '東京メトロ半蔵門線/押上駅', '6', '10', '10', '6', '7', '5', '16.7', '2DK', '42.21', 'https://img01.suumo.com/front/gazo/fr/bukken/703/100362266703/100362266703_gw.jpg')
('パークアクシス押上レジデンス', '東京都墨田区向島３', '東武伊勢崎線/とうきょうスカイツリー駅', '都営浅草線/本所吾妻橋駅', '東京メトロ半蔵門線/押上駅', '6'